<h1> Battle of the Neighborhoods (Austin Food Tour): </h1>

<h2> 1. Introduction/Business Problem:</h2>

<p style="font-size:18px">	Travel and tourism is one of Texas’s biggest economic drivers, generating $80.2 billion in direct spending in 2018 [1]. People from all over the world flock to experience the unique culture and experience that the Lone Star State has to offer. Austin is one of the most tourist-friendly and famous cities in Texas. Austin hosts some of the most popular music festivals in the country like SXSW and the Austin City Limits Music Festival and is also home to the Circuit of The Americas. Because of the breadth of activities and experiences people can partake in, Austin draws in approximately 24.7 million domestic visitors each year [2].</p>
	<p style="font-size:18px">Like many other cities, Austin is also home to some incredible restaurants. Tourists can indulge in unique cuisines all over the city and make their trip even more memorable. In this analysis, we are seeking to cluster the various neighborhoods in Austin by the types of restaurants they most frequently offer to understand what regions of Austin offer the most options of a specific type of cuisine. Then, we will be identifying the 5 most highly rated restaurants of the most popular cuisine for each cluster of neighborhoods in Austin to create a “must-go” restaurant list. </p>
	<p style="font-size:18px">The stakeholders of this project are the city of Austin and its tourism board as this project will drive heavier tourist traffic around different areas of Austin. Travel bloggers, tourists, and college students will also benefit from this project as they can go to restaurants that are well-liked and experience cuisines that a particular neighborhood is known for. Restaurant owners have a great stake in this project as well because their restaurants may experience heavy foot-traffic because they were represented on this list. Restaurant owners who don’t rank as well may also use this listing and work to improve their rating.</p>

<h2>2. Data</h2>

<p style="font-size:18px"> The data required for this project will be collected via web-scraping and by using Foursquare’s API. Web-scraping will be used to create a pandas DataFrame of all of the neighborhoods available in Central Austin. We will then get the Latitude and Longitude values of the neighborhoods using the Nominatim package in python and add those values to our DataFrame. The final data we will be using is location data from Foursquare. We will be querying Foursquare’s API for data on what specific restaurants exist within Austin’s neighborhoods and how well they are rated. This information will be used for determining which cuisines are most popular in which neighborhoods and will allow us to cluster them according to similarity. We will then be able to complete our list of the 5 most highly rated restaurants in each cluster of neighborhoods.</p>

<h3>2.1 Analysis</h3>
<p style="font-size:18px">
To begin our analysis, we will be scraping Wikipedia for a list of all Neighborhoods in Central Austin. We will do this using the requests and BeautifulSoup libraries and then transport the information to a pandas dataframe. To do this, we are going to be importing all of the libraries we will need.</p>


In [8]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
!pip install geocoder
!pip install folium
!pip install geopy

import requests
import pandas as pd
import geocoder
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import numpy as np
import geopy

from bs4 import BeautifulSoup as bsoup
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

print('Packages fully loaded and installed.')

Packages fully loaded and installed.


<p style="font-size:18px">Then, we will begin our web-scraping by defining the URL we will be using data from. We will go ahead and extract the data we need using the steps outlined below.</p>

In [9]:
#define URL source
source = requests.get('https://en.wikipedia.org/wiki/List_of_Austin_neighborhoods').text

#create a Beautiful Soup object and define lxml as the parser
soup= bsoup(source, 'lxml')

#isolate the Austin Neighborhood names and add them to an empty list
table= soup.find('table',class_ = 'wikitable')
tabledata = table.tbody.text.split('\n\n')
tabledata
tableitems = []

#append items to table data
for rows in tabledata:
    temp  = rows.split('\n')[1:]
    if (temp != []):
        tableitems.append(temp[0])

df=pd.DataFrame(tableitems)
new_header = df.iloc[0]
df = df[1:] 
df.columns = new_header
df.rename(columns={'Name': "Neighborhood"}, inplace=True)
df

,Neighborhood
1,Bryker Woods
2,Caswell Heights
3,Downtown Austin
4,Eastwoods
5,Hancock
6,Heritage
7,Hyde Park
8,Judges' Hill
9,Lower Waller Creek
10,North University


<p style="font-size:18px">Next, use the Nominatim Package to check the Latitude and Longitude coordinates to the DataFrame. Since Wikipedia is editable by any user, we want to make sure that all of the neighborhoods we scraped have coordinates present in Nominatim and that it does not return 'None'.</p>


In [10]:
#create a function to handle TimeOuts from Geocoder
from geopy.exc import GeocoderTimedOut
locator = Nominatim(user_agent = "austinagent")
def do_geocode(address):
    try:
        return locator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [11]:
neighborhoods = df.values.tolist()

#check if each neighborhood is returning data from Nominatim
for neighborhood in neighborhoods:
    g = do_geocode('{}, Austin, Texas'.format(neighborhood))
    if (g == None):
        print(str(neighborhood) + ': None')
    else:
        print(str(neighborhood) + ': ' + str(g.latitude) + ', ' + str(g.longitude))

['Bryker Woods']: 30.305246, -97.7545846
['Caswell Heights']: None
['Downtown Austin']: 30.2680536, -97.7447642
['Eastwoods']: 30.290562350000002, -97.73141774152433
['Hancock']: 30.2958956, -97.7247678
['Heritage']: 30.3457961, -97.6909909
['Hyde Park']: 30.3044117, -97.7304485
["Judges' Hill"]: None
['Lower Waller Creek']: None
['North University']: 30.42100665, -97.84008580625134
['Oakmont Heights']: 30.3119891, -97.7540697
['Old Enfield']: 30.2848643, -97.7591064
['Old Pecan Street']: 29.8905618, -96.4887858
['Old West Austin']: 30.296822, -97.7548514
['Original Austin']: 30.2737783, -97.72048
['Original West University']: None
['Pemberton Heights']: 30.1900716, -97.83857107484684
['Ridgelea']: 30.3115548, -97.7513953
['Ridgetop']: 30.311527050000002, -97.71641825831956
['Rosedale']: 30.3133901, -97.7448982
['Shoal Crest']: 30.2974652, -97.7478879
['West Downtown']: 30.2680536, -97.7447642


<p style="font-size:18px">As you can see, some neighborhoods in our DataFrame are returning 'None' when Nominatim tries to find the coordinates for these locations. After further investigation, we found that Nominatim does not have coordinates for these neighborhoods, as shown in the screenshot below. For the purposes of this exercise, we will be either dropping or renaming the rows which do not have coordinates in Nominatim. The Neighborhoods that do not have coordinates are "Caswell Heights", "Judges' Hill", "Lower Waller Creek", and "Original West University". Nominatim does not have data on a "Lower Waller Creek", but it does have data on "Waller Creek". For this reason, we will be renaming this neighborhood, but we will be dropping the rest.</p>


In [12]:
#replace neighborhood
df.replace(['Lower Waller Creek'],['Waller Creek'], inplace=True)

#add coordinate items to list with new dataframe
neighborhoods = df.values.tolist()
latitude = []
longitude = []
for neighborhood in neighborhoods:
    g = do_geocode('{}, Austin, Texas'.format(neighborhood))
    if (g == None):
        latitude.append('0')
        longitude.append('0')
    else:
        latitude.append(g.latitude)
        longitude.append(g.longitude)

#add coordinates column to dataframe
df['Latitude'] = latitude
df['Longitude'] = longitude

#remove neighborhoods with no coordinates
df= df[df.Latitude!='0']
df= df.reset_index()
df.drop(columns = ['index'], axis = 1, inplace=True)
#df.drop(columns = ['level_0'], axis = 1, inplace=True)
df

,Neighborhood,Latitude,Longitude
0,Bryker Woods,30.3052,-97.7546
1,Downtown Austin,30.2681,-97.7448
2,Eastwoods,30.2906,-97.7314
3,Hancock,30.2959,-97.7248
4,Heritage,30.3458,-97.691
5,Hyde Park,30.3044,-97.7304
6,Waller Creek,30.2821,-97.7337
7,North University,30.421,-97.8401
8,Oakmont Heights,30.312,-97.7541
9,Old Enfield,30.2849,-97.7591


<p style="font-size:18px">We have cleaned up our neighborhood DataFrame and removed all neighborhoods that do not return any coordinates from Nominatim. We are now ready to use the Foursquare API to get our restaurant data.</p>

<h2>Sources</h2>
<ol style="font-size:18px"> <li>https://www.bizjournals.com/austin/news/2019/05/02/tourisms-economic-impact-in-texas-164-billion.html</li>
<li>https://www.austintexas.org/travel-professionals/</li></ol>